In [1]:
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

In [2]:
def print_matrix(M: np.ndarray):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    """
    for row in M:
        print(f"{np.array2string(row, precision=2 ,formatter={'float': lambda x: f'{x:.2f}'},separator=', ')}")
    print('\n')

def print_data_ext_ps(V1: np.complex_, V2: np.complex_):
    """
    Function to print data on the effect of the external phaseshift
    that's used to match the phases of two given elements.
    
    Parameters
    ------
    V1 : element of the auxillary matrix `V`
    V2 : subsequent element of **V1**
    
    ---
    
    Further description:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    """
    
    eq_stat = (np.angle(V1) == np.angle(V2))
    eq_str = "\nThey are matching!"
    neq_str = "\nThey are NOT matching!\n"

    print('Affected elements:\t',
        "{:.2f}, {:.2f}".format(V1, V2))
    print('Corresponding angles:\t',
        "{:.2f}, {:.2f}".format(np.angle(V1), np.angle(V2)))

    if eq_stat:
        print(eq_str)
    else:
        print(neq_str)
        print('Full length of angles:\n',
            "{}, {}".format(np.angle(V1), np.angle(V2)))
        print('\nTheir difference:\t',
            "{}".format(np.angle(V1) - np.angle(V2)))

In [3]:
U = itf.random_unitary(5)

# Odd diagonals

### Finding external phaseshifts
They match the given elements' phases

In [4]:
V = np.conjugate(U.T)
# odd diags: 1,3,5...
m = U.shape[0]
j = 1

x = m-1
y = j-1

P = itf.external_ps(m, j, V[x,y], V[x,y+1])
print('old V:\n')
print_matrix(V)
V = np.matmul(V,P)
print("NOTE: {}. column was affected!".format(j+1))
print('new V:\n')
print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

old V:

[ 0.47-0.26j,  0.49+0.j  ,  0.3 -0.15j,  0.  -0.51j, -0.31+0.07j]
[-0.06+0.49j,  0.31+0.23j, -0.26-0.31j, -0.03-0.23j,  0.37+0.5j ]
[ 0.13+0.08j, -0.03+0.39j, -0.37-0.46j, -0.42+0.15j, -0.37-0.39j]
[-0.39-0.24j, -0.01+0.2j , -0.45+0.32j,  0.05-0.63j, -0.01-0.21j]
[ 0.48-0.04j, -0.63+0.17j, -0.11+0.24j, -0.24-0.19j, -0.05+0.43j]


NOTE: 2. column was affected!
new V:

[ 0.47-0.26j, -0.48-0.1j ,  0.3 -0.15j,  0.  -0.51j, -0.31+0.07j]
[-0.06+0.49j, -0.26-0.29j, -0.26-0.31j, -0.03-0.23j,  0.37+0.5j ]
[ 0.13+0.08j,  0.1 -0.38j, -0.37-0.46j, -0.42+0.15j, -0.37-0.39j]
[-0.39-0.24j,  0.05-0.19j, -0.45+0.32j,  0.05-0.63j, -0.01-0.21j]
[ 0.48-0.04j,  0.65-0.05j, -0.11+0.24j, -0.24-0.19j, -0.05+0.43j]


Affected elements:	 0.48-0.04j, 0.65-0.05j
Corresponding angles:	 -0.07, -0.07

They are NOT matching!

Full length of angles:
 -0.07481848347792765, -0.07481848347792763

Their difference:	 -1.3877787807814457e-17


In [5]:
# these are the elements which angles need to be matched by summa (?)
print_data_ext_ps(V[x-1,y],V[x-1,y+1])

Affected elements:	 -0.39-0.24j, 0.05-0.19j
Corresponding angles:	 -2.60, -1.34

They are NOT matching!

Full length of angles:
 -2.5990761244296943, -1.3379966262556047

Their difference:	 -1.2610794981740896


## Finding internal phaseshifts
$\delta$ and $\sum$

In [6]:
# looking at odd diagonal version!

delta = itf.custom_arctan(V[x,y], V[x,y+1])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))


# summ = (np.angle(V[x-1,y]) + np.angle(V[x,y]))/2
# summ = np.angle(V[x-1,y]) - np.angle(V[x-1,y+1])
summ = 0

# might need to change into 'k' dependence
modes = [j-1, j]    # initial mode-pairs
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

print('\nBlock of M2:\n')
print_matrix(M[j-1:j+1,j-1:j+1])


print('old V:\n')
print_matrix(V)
V = np.matmul(V,M)
print("NOTE: {}. and {}. column were affected!".format(j,j+1))
print('new V:\n')
print_matrix(V)

delta:	-0.64+0.00j	 -> real: False
angle:	180.0°

Block of M2:

[-0.59+5.52e-18j,  0.8 +4.09e-18j]
[0.8 +4.09e-18j, 0.59-5.52e-18j]


old V:

[ 0.47-0.26j, -0.48-0.1j ,  0.3 -0.15j,  0.  -0.51j, -0.31+0.07j]
[-0.06+0.49j, -0.26-0.29j, -0.26-0.31j, -0.03-0.23j,  0.37+0.5j ]
[ 0.13+0.08j,  0.1 -0.38j, -0.37-0.46j, -0.42+0.15j, -0.37-0.39j]
[-0.39-0.24j,  0.05-0.19j, -0.45+0.32j,  0.05-0.63j, -0.01-0.21j]
[ 0.48-0.04j,  0.65-0.05j, -0.11+0.24j, -0.24-0.19j, -0.05+0.43j]


NOTE: 1. and 2. column were affected!
new V:

[-6.62e-01+0.08j,  9.34e-02-0.27j,  3.02e-01-0.15j,  6.04e-04-0.51j,
 -3.11e-01+0.07j]
[-0.17-0.52j, -0.2 +0.22j, -0.26-0.31j, -0.03-0.23j,  0.37+0.5j ]
[ 0.01-0.35j,  0.17-0.16j, -0.37-0.46j, -0.42+0.15j, -0.37-0.39j]
[ 0.27-0.01j, -0.29-0.3j , -0.45+0.32j,  0.05-0.63j, -0.01-0.21j]
[ 0.24-0.02j,  0.77-0.06j, -0.11+0.24j, -0.24-0.19j, -0.05+0.43j]




In [7]:
print(V[x,y])
print(np.abs(V[x,y]), np.angle(V[x,y],True))

(0.23569684052142736-0.017667458810044664j)
0.23635807524302566 -4.286783332854533


## Even diagonals

In [58]:
V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0

P = itf.external_ps(m, j, V[x,y], V[x-1,y])
print('old V:\n')
print_matrix(V)
V = np.matmul(P,V)
print("NOTE: {}. row was affected!".format(j+1))
print('new V:\n')
print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

old V:

[-0.77+0.17j, -0.28+0.05j, -0.09-0.24j,  0.27+0.3j , -0.27+0.01j]
[ 0.12+0.34j, -0.35+0.15j,  0.18-0.j  , -0.43+0.56j,  0.43+0.06j]
[-0.21+0.03j,  0.14+0.1j ,  0.79+0.46j,  0.28-0.03j,  0.09+0.11j]
[-0.27-0.12j,  0.28+0.79j, -0.02-0.06j, -0.4 -0.2j , -0.  -0.12j]
[-0.26-0.23j,  0.2 -0.05j, -0.24-0.1j ,  0.24-0.03j,  0.78+0.31j]


NOTE: 3. row was affected!
new V:

[-0.77+0.17j, -0.28+0.05j, -0.09-0.24j,  0.27+0.3j , -0.27+0.01j]
[ 0.12+0.34j, -0.35+0.15j,  0.18-0.j  , -0.43+0.56j,  0.43+0.06j]
[-0.19-0.09j,  0.06+0.16j,  0.4 +0.81j,  0.25+0.13j,  0.01+0.14j]
[-0.27-0.12j,  0.28+0.79j, -0.02-0.06j, -0.4 -0.2j , -0.  -0.12j]
[-0.26-0.23j,  0.2 -0.05j, -0.24-0.1j ,  0.24-0.03j,  0.78+0.31j]


Affected elements:	 -0.27-0.12j, -0.19-0.09j
Corresponding angles:	 -2.71, -2.71

They are NOT matching!

Full length of angles:
 -2.7141252871628736, -2.714125287162873

Their difference:	 -4.440892098500626e-16
